In [1]:
from kafka import KafkaProducer
from kafka.errors import KafkaError, KafkaTimeoutError
import atexit
import logging
import json
from pymongo import MongoClient

In [2]:
logger_format = '%(asctime)-15s %(message)s'
logging.basicConfig(format=logger_format)
logger = logging.getLogger('test')
logger.setLevel(logging.DEBUG)

In [5]:
topic_name = 'stockTopic'
kafka_broker = '127.0.0.1:9092'
mongo_client='localhost:27017'
db_name='stock_test'
collection_name='stock_price_test'

In [6]:
def dump_data(stock_data,collection):
  '''
  function to store data in MongoDB from Kafka
  :param data: stock data json
  :param collection: mongo collection session
  :return: None
  '''

  try:
    logger.info(f'Dumping data in MongoDB {json.dumps(stock_data)}')
    collection.insert_one(stock_data)
    logger.info(f'Data dumped in MongoDB {json.dumps(stock_data)}')
  except KafkaError as ke:
    logger.warning(f'Kafka Error {ke}')
  except Exception as e:
    logger.warning(f'Failed to fetch stock price of {symbol} caused by {e}')

In [1]:
def on_shutdown(consumer,mongo_session):
  '''
  shutdown hook to be called before the shutdown
  :param consumer: instance of a kafka consumer
  :return: None
  '''
  try:
    logger.info('Closing Kafka Consumer')
    consumer.close()
    logger.info('Kafka Consumer Close')
  except KafkaError as ke:
    logger.warning(f'Failed to close Kafka Consumer, due to {ke}')
  finally:
      logger.info("Consumer Session closed")

In [ ]:
def createCollectionMongo():
    '''
    function to create mongo session which will be used to write data
    to mongo client
    :return: none
    '''
    client = MongoClient(mongo_client)
    db = client[db_name]
    collection = client[collection_name]
    collection.create_index('StockSymbol', name="stock_symbol_index")
    return collection

In [8]:
if __name__=='__main__':
    consumer=KafkaConsumer(
        bootstrap_servers=kafka_broker,
        value_deserializer=lambda x: json.loads(x).decode('utf-8')
    )
    
    collection_session=createCollectionMongo()
    
    atexit.register(on_shutdown,consumer)
    for msg in consumer:
        data=msg.value
        data['_id']=f"{data['StockSymbol']}_{data['Time']}"
        dump_data(msg.value,collection_session)

2021-11-25 19:28:00,395 Fetching stock price of GC=F
2021-11-25 19:28:06,293 Received stock price of GC=F at 2021-11-25T19:28:06Z : 1792.4
2021-11-25 19:28:07,299 Fetching stock price of GC=F
2021-11-25 19:28:12,400 Received stock price of GC=F at 2021-11-25T19:28:12Z : 1792.4
2021-11-25 19:28:13,410 Fetching stock price of GC=F
2021-11-25 19:28:19,542 Received stock price of GC=F at 2021-11-25T19:28:19Z : 1792.4
2021-11-25 19:28:20,556 Fetching stock price of GC=F
2021-11-25 19:28:25,798 Received stock price of GC=F at 2021-11-25T19:28:25Z : 1792.4
2021-11-25 19:28:26,813 Fetching stock price of GC=F
2021-11-25 19:28:32,296 Received stock price of GC=F at 2021-11-25T19:28:32Z : 1792.4
2021-11-25 19:28:33,308 Fetching stock price of GC=F
2021-11-25 19:28:38,709 Received stock price of GC=F at 2021-11-25T19:28:38Z : 1792.4
2021-11-25 19:28:39,718 Fetching stock price of GC=F
2021-11-25 19:28:44,888 Received stock price of GC=F at 2021-11-25T19:28:44Z : 1792.4
2021-11-25 19:28:45,897 Fet

KeyboardInterrupt: 